## DATA EXTRACTION

### Extraction sources:

In [ ]:
# Local files → COSING_Annex_III_v2.xls, product_info.csv, cosmetics.csv
# REST API → Open Beauty Facts API (cosmetic products, JSON format)
# Web scraping → Wikipedia HTML page (HTML parsing with BeautifulSoup)
# MySQL database → sephora_products / skincare_products tables already imported

In [1]:
import sys
import os
import logging
import traceback
from datetime import datetime
from pathlib import Path

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

In [3]:
# Load environment variables (.env) 
load_dotenv()

True

In [4]:
# Path configuration 
BASE_DIR      = Path.cwd()
RAW_DIR       = BASE_DIR / "data" / "raw"
PROCESSED_DIR = BASE_DIR / "data" / "processed"
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

In [5]:
# Logging configuration
LOG_FILE = BASE_DIR / "logs" / "extraction.log"
LOG_FILE.parent.mkdir(parents=True, exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[
        logging.FileHandler(LOG_FILE, encoding="utf-8"),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

In [6]:
# Read parameters from .env
MYSQL_USER     = os.getenv("MYSQL_USER",     "root")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD", "")
MYSQL_HOST     = os.getenv("MYSQL_HOST",     "localhost")
MYSQL_PORT     = os.getenv("MYSQL_PORT",     "3306")
MYSQL_DB       = os.getenv("MYSQL_DB",       "cosmetics_regulatory_db")

In [7]:
# Local file paths
FILE_COSING   = os.getenv("FILE_COSING",   str(BASE_DIR / "data" / "raw" / "COSING_Annex_III_v2.xls"))
FILE_SEPHORA  = os.getenv("FILE_SEPHORA",  str(BASE_DIR / "data" / "raw" / "product_info.csv"))
FILE_SKINCARE = os.getenv("FILE_SKINCARE", str(BASE_DIR / "data" / "raw" / "cosmetics.csv"))

In [8]:
# URL for the REST API and web scraping
API_OPEN_BEAUTY_FACTS_URL      = "https://world.openfoodfacts.org/cgi/search.pl"
SCRAPING_WIKIPEDIA_COSM_URL    = "https://en.wikipedia.org/wiki/Ingredients_of_cosmetics"

### 1. EXTRACTION FROM LOCAL FILES

In [9]:
def extract_from_files() -> dict[str, pd.DataFrame]:
    """
    Reads the three local raw data files.
    Returns a dictionary {source_name: DataFrame}.
    """
    logger.info("── Source 1: Extraction from local files ──")
    results = {}

    # 1a. COSING (Excel file)
    try:
        logger.info(f"  Reading {FILE_COSING} ...")
        df = pd.read_excel(FILE_COSING, engine="xlrd")
        results["cosing"] = df
        logger.info(f"  ✓ COSING loaded: {df.shape[0]} rows, {df.shape[1]} columns")
    except FileNotFoundError:
        logger.error(f"  ✗ File not found: {FILE_COSING}")
        results["cosing"] = pd.DataFrame()
    except Exception as e:
        logger.error(f"  ✗ Error reading COSING: {e}")
        results["cosing"] = pd.DataFrame()

    # 1b. SEPHORA (CSV file)
    try:
        logger.info(f"  Reading {FILE_SEPHORA} ...")
        df = pd.read_csv(FILE_SEPHORA)
        results["sephora"] = df
        logger.info(f"  ✓ SEPHORA loaded: {df.shape[0]} rows, {df.shape[1]} columns")
    except FileNotFoundError:
        logger.error(f"  ✗ File not found: {FILE_SEPHORA}")
        results["sephora"] = pd.DataFrame()
    except Exception as e:
        logger.error(f"  ✗ Error reading SEPHORA: {e}")
        results["sephora"] = pd.DataFrame()

    # 1c. SKINCARE (CSV file)
    try:
        logger.info(f"  Reading {FILE_SKINCARE} ...")
        df = pd.read_csv(FILE_SKINCARE)
        results["skincare"] = df
        logger.info(f"  ✓ SKINCARE loaded: {df.shape[0]} rows, {df.shape[1]} columns")
    except FileNotFoundError:
        logger.error(f"  ✗ File not found: {FILE_SKINCARE}")
        results["skincare"] = pd.DataFrame()
    except Exception as e:
        logger.error(f"  ✗ Error reading SKINCARE: {e}")
        results["skincare"] = pd.DataFrame()

    return results

### 2. EXTRACTION VIA REST API (Open Beauty Facts — JSON)

In [10]:
def extract_from_api(max_pages: int = 3) -> pd.DataFrame:
    """
    Calls the Open Beauty Facts API with json=1 → retrieves JSON directly.
    Parameter:
        max_pages: number of result pages to fetch.
    Returns a DataFrame with the extracted products.
    """
    logger.info("── Source 2: Extraction via REST API (Open Beauty Facts — JSON) ──")

    all_products = []
    headers = {"User-Agent": "Cosmetics-Regulatory-Project/1.0 (educational)"}

    for page in range(1, max_pages + 1):
        try:
            logger.info(f"  API request page {page}/{max_pages} ...")
            params = {
                "search_terms": "cosmetics",
                "search_simple": "1",
                "action": "process",
                "json": "1",            # ← request JSON
                "page": page,
                "count": 25
            }
            response = requests.get(
                API_OPEN_BEAUTY_FACTS_URL,
                params=params,
                headers=headers,
                timeout=30
            )
            response.raise_for_status()

            data = response.json()
            products = data.get("products", [])

            if not products:
                logger.info(f"  No products on page {page}, stopping pagination.")
                break

            for product in products:
                all_products.append({
                    "product_name":     product.get("product_name", ""),
                    "brand":            product.get("brands", ""),
                    "ingredients_text": product.get("ingredients_text", ""),
                    "categories":       product.get("categories", ""),
                    "country":          product.get("countries", ""),
                    "barcode":          product.get("code", ""),
                    "source":           "open_beauty_facts_api"
                })

            logger.info(f"  ✓ Page {page}: {len(products)} products retrieved")

        except requests.exceptions.Timeout:
            logger.warning(f"  ⚠ Timeout on page {page} — continuing with next page.")
            continue
        except requests.exceptions.HTTPError as e:
            logger.error(f"  ✗ HTTP error page {page}: {e}")
            continue
        except requests.exceptions.RequestException as e:
            logger.error(f"  ✗ Network error page {page}: {e}")
            break
        except (ValueError, KeyError) as e:
            logger.error(f"  ✗ JSON parsing error page {page}: {e}")
            continue

    df = pd.DataFrame(all_products)
    logger.info(f"  ✓ REST API — total: {len(df)} products extracted")
    return df

### 3. EXTRACTION VIA WEB SCRAPING (Wikipedia — Ingredients of cosmetics)

In [11]:
def extract_by_scraping() -> pd.DataFrame:
    """
    Scrape the Wikipedia page "Ingredients of cosmetics" to extract
    the list of common cosmetic ingredients and their links.

    Confirmed DOM structure:
        - The heading is <h2 id="Common_ingredients"> inside a
          <div class="mw-heading mw-heading2">
        - The grandparent of this <div> contains all <p> elements of the section
        - Each <p> contains <a href="/wiki/..."> links to ingredients
        - Stop when the next <div class="mw-heading mw-heading2"> is encountered
          (which is "Types of cosmetics")

    Returns a DataFrame with: ingredient, wikipedia_link, description, source
    """
    logger.info("── Source 3: Extraction via Web Scraping (Wikipedia — Ingredients of cosmetics) ──")

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        )
    }

    scraped_ingredients = []

    try:
        logger.info(f"  GET request to {SCRAPING_WIKIPEDIA_COSM_URL} ...")
        response = requests.get(SCRAPING_WIKIPEDIA_COSM_URL, headers=headers, timeout=30)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        logger.info(f"  ✓ HTML page retrieved — size: {len(response.text)} characters")

        # ── Find the "Common_ingredients" heading ──
        heading_h2 = soup.find("h2", {"id": "Common_ingredients"})
        if not heading_h2:
            logger.warning("  ⚠ Section 'Common_ingredients' not found on the page")
            return pd.DataFrame(columns=["ingredient", "wikipedia_link", "description", "source"])

        # ── Navigate to the grandparent containing the content ──
        # heading_h2 is inside a <div class="mw-heading mw-heading2">
        # the grandparent of this div contains all <p> elements of the section
        heading_div = heading_h2.parent   # <div class="mw-heading mw-heading2">
        container   = heading_div.parent  # container with all <p>

        logger.info(f"  Container found: <{container.name}>")

        # ── Iterate over the children of the container after heading_div ──
        found_heading = False
        for child in container.children:
            # Start collecting after the "Common_ingredients" heading
            if child is heading_div:
                found_heading = True
                continue

            if not found_heading:
                continue

            # Stop at the next <div class="mw-heading mw-heading2"> (= next section)
            if child.name == "div" and "mw-heading2" in child.get("class", []):
                logger.info(f"  End of section detected: '{child.get_text(strip=True)[:40]}'")
                break

            # Process only <p> (paragraphs of content)
            if child.name == "p":
                paragraph_text = child.get_text(strip=True)

                # Extract all <a href="/wiki/..."> links in this paragraph
                for a in child.find_all("a", href=True):
                    href = a["href"]
                    # Keep only internal Wikipedia links (/wiki/...) and exclude special links
                    if href.startswith("/wiki/") and not href.startswith("/wiki/Special:"):
                        ingredient_name = a.get_text(strip=True)
                        # Filter: ignore very short or generic texts
                        if len(ingredient_name) > 2:
                            scraped_ingredients.append({
                                "ingredient":      ingredient_name.upper(),
                                "wikipedia_link":  "https://en.wikipedia.org" + href,
                                "description":     paragraph_text[:200],
                                "source":          "wikipedia_scraping"
                            })

        # ── Deduplicate by ingredient name ──
        df = pd.DataFrame(scraped_ingredients)
        if not df.empty:
            df = df.drop_duplicates(subset=["ingredient"]).reset_index(drop=True)

        logger.info(f"  ✓ Scraping — total: {len(df)} ingredients extracted")
        return df

    except requests.exceptions.Timeout:
        logger.error("  ✗ Timeout during Wikipedia scraping")
        return pd.DataFrame(columns=["ingredient", "wikipedia_link", "description", "source"])
    except requests.exceptions.HTTPError as e:
        logger.error(f"  ✗ HTTP error during scraping: {e}")
        return pd.DataFrame(columns=["ingredient", "wikipedia_link", "description", "source"])
    except requests.exceptions.RequestException as e:
        logger.error(f"  ✗ Network error during scraping: {e}")
        return pd.DataFrame(columns=["ingredient", "wikipedia_link", "description", "source"])
    except Exception as e:
        logger.error(f"  ✗ Unexpected error during scraping: {e}")
        return pd.DataFrame(columns=["ingredient", "wikipedia_link", "description", "source"])


### 4. EXTRACTION FROM THE MySQL DATABASE

In [12]:
def extract_from_database() -> dict[str, pd.DataFrame]:
    """
    Connects to the MySQL database and extracts data from the
    sephora_products and skincare_products tables.
    Returns a dictionary {table_name: DataFrame}.
    """
    logger.info("── Source 4: Extraction from the MySQL database ──")

    connection_string = (
        f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}"
        f"@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"
    )
    results = {}

    try:
        logger.info(f"  Connecting to MySQL: {MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB} ...")
        engine = create_engine(connection_string, pool_pre_ping=True)

        # ── Test connection ──
        with engine.connect() as conn:
            conn.execute(text("SELECT 1"))
        logger.info("  ✓ MySQL connection successful")

        # ── Extract sephora_products table ──
        try:
            query_sephora = text("""
                SELECT
                    product_id,
                    product_name,
                    brand_name,
                    product_type,
                    price_usd,
                    rating,
                    has_restricted_ingredient,
                    has_cmr,
                    restricted_ingredient_count,
                    cmr_count
                FROM sephora_products
                WHERE product_name IS NOT NULL
                ORDER BY product_id
            """)
            df_sephora = pd.read_sql(query_sephora, engine)
            results["db_sephora"] = df_sephora
            logger.info(f"  ✓ sephora_products: {len(df_sephora)} rows extracted")
        except Exception as e:
            logger.error(f"  ✗ Error extracting sephora_products: {e}")
            results["db_sephora"] = pd.DataFrame()

        # ── Extract skincare_products table ──
        try:
            query_skincare = text("""
                SELECT
                    brand,
                    product_name,
                    product_type,
                    price,
                    rating,
                    has_restricted_ingredient,
                    has_cmr,
                    restricted_ingredient_count,
                    cmr_count
                FROM skincare_products
                WHERE product_name IS NOT NULL
                ORDER BY brand, product_name
            """)
            df_skincare = pd.read_sql(query_skincare, engine)
            results["db_skincare"] = df_skincare
            logger.info(f"  ✓ skincare_products: {len(df_skincare)} rows extracted")
        except Exception as e:
            logger.error(f"  ✗ Error extracting skincare_products: {e}")
            results["db_skincare"] = pd.DataFrame()

        engine.dispose()

    except Exception as e:
        logger.error(f"  ✗ Unable to connect to MySQL: {e}")
        logger.info("  ⚠ MySQL source ignored — script continues with other sources.")
        results["db_sephora"]  = pd.DataFrame()
        results["db_skincare"] = pd.DataFrame()

    return results


### 5. SAVE RAW EXTRACTED DATA

In [13]:
def save_raw(name: str, df: pd.DataFrame) -> None:
    """Save a raw DataFrame into the data/raw folder with a timestamp."""
    if df.empty:
        logger.warning(f"  ⚠ DataFrame '{name}' is empty — nothing saved.")
        return
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = RAW_DIR / f"{name}_{timestamp}.csv"
    df.to_csv(filepath, index=False, encoding="utf-8")
    logger.info(f"  💾 Saved: {filepath} ({len(df)} rows)")

### 6. MAIN ENTRY POINT

In [14]:
def main():
    """
    Entry point of the script.
    Orchestrates extraction from the 4 sources,
    saves the raw results, and produces a summary report.
    """
    start_time = datetime.now()
    logger.info("=" * 60)
    logger.info("  START OF EXTRACTION — " + start_time.strftime("%Y-%m-%d %H:%M:%S"))
    logger.info("=" * 60)

    extraction_report = {}  # for the final report

    # ── 1. Local files ──
    try:
        files_data = extract_from_files()
        for name, df in files_data.items():
            save_raw(name, df)
            extraction_report[f"file_{name}"] = len(df)
    except Exception as e:
        logger.error(f"Unhandled error in extract_from_files: {e}\n{traceback.format_exc()}")

    # ── 2. REST API ──
    try:
        api_data = extract_from_api(max_pages=3)
        save_raw("api_open_beauty_facts", api_data)
        extraction_report["api_open_beauty_facts"] = len(api_data)
    except Exception as e:
        logger.error(f"Unhandled error in extract_from_api: {e}\n{traceback.format_exc()}")

    # ── 3. Web Scraping ──
    try:
        scraping_data = extract_by_scraping()
        save_raw("scraping_wikipedia_cosmetics", scraping_data)
        extraction_report["scraping_wikipedia"] = len(scraping_data)
    except Exception as e:
        logger.error(f"Unhandled error in extract_by_scraping: {e}\n{traceback.format_exc()}")

    # ── 4. MySQL Database ──
    try:
        db_data = extract_from_database()
        for name, df in db_data.items():
            save_raw(name, df)
            extraction_report[name] = len(df)
    except Exception as e:
        logger.error(f"Unhandled error in extract_from_database: {e}\n{traceback.format_exc()}")

    # ── Summary Report ──
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()

    logger.info("=" * 60)
    logger.info("  EXTRACTION REPORT")
    logger.info("=" * 60)
    for source, count in extraction_report.items():
        status = "✓" if count > 0 else "✗ (empty)"
        logger.info(f"  {status}  {source:.<40} {count} rows")
    logger.info(f"  Total duration: {duration:.2f} seconds")
    logger.info("=" * 60)


# ── Execution ──
if __name__ == "__main__":
    main()


2026-02-06 21:22:13 [INFO] ============================================================
2026-02-06 21:22:13 [INFO]   START OF EXTRACTION — 2026-02-06 21:22:13
2026-02-06 21:22:13 [INFO] ============================================================
2026-02-06 21:22:13 [INFO] ── Source 1: Extraction from local files ──
2026-02-06 21:22:13 [INFO]   Reading data/raw/COSING_Annex_III_v2.xls ...
2026-02-06 21:22:13 [INFO]   ✓ COSING loaded: 375 rows, 16 columns
2026-02-06 21:22:13 [INFO]   Reading data/raw/product_info.csv ...
2026-02-06 21:22:13 [INFO]   ✓ SEPHORA loaded: 8494 rows, 27 columns
2026-02-06 21:22:13 [INFO]   Reading data/raw/cosmetics.csv ...
2026-02-06 21:22:13 [INFO]   ✓ SKINCARE loaded: 1472 rows, 11 columns
2026-02-06 21:22:13 [INFO]   💾 Saved: C:\Users\ZINA\Desktop\IRONHACK\Week_8\cosmetics\data\raw\cosing_20260206_212213.csv (375 rows)
2026-02-06 21:22:14 [INFO]   💾 Saved: C:\Users\ZINA\Desktop\IRONHACK\Week_8\cosmetics\data\raw\sephora_20260206_212213.csv (8494 rows)
202